In [58]:
from keras.models import Sequential , Model
from keras.layers import Dense ,  BatchNormalization , Reshape , Input , Flatten
from keras.layers import Conv2D , MaxPool2D , Conv2DTranspose , UpSampling2D , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation
from keras.layers import Dropout

from keras.layers import Add , Multiply

from keras.initializers import random_normal

from keras.optimizers import Adam

from keras.datasets import mnist

In [59]:
import os

import matplotlib as plt
import numpy as np

import keras.backend as K

import gc

%matplotlib inline

In [60]:
WIDTH = 28
HEIGHT = 28
CHANNEL = 1

LATENT_DIM = 100 #latent variable z sample from normal distribution

BATCH_SIZE = 128
EPOCHS = 10

PATH = 'faces/'

#生成多少个图像 长*宽
ROW = 5
COL = 5


In [61]:
'''
load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)
'''

'\nload_index = 0\n\nimages_name = os.listdir(PATH)\n\nIMAGES_COUNT = len(images_name)\n'

In [62]:

(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)


In [63]:

def load_mnist():
    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = decoder_i.predict(noise)
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('mnist_aae/No.%d.png' % epoch)
    plt.pyplot.close()



In [64]:
'''
def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
    
    load_index += batch_size
    
    return np.array(images)/127.5-1

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = (generated_image+1)*127.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('generated_faces_dcgan/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')
'''

"\ndef load_image(batch_size = BATCH_SIZE):\n    global load_index\n    \n    images = []\n    \n    for i in range(batch_size):\n        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))\n    \n    load_index += batch_size\n    \n    return np.array(images)/127.5-1\n\ndef write_image(epoch):\n    \n    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))\n    generated_image = generator_i.predict(noise)\n    generated_image = (generated_image+1)*127.5\n    \n    fig , axes = plt.pyplot.subplots(ROW , COL)\n    \n    count=0\n    \n    for i in range(ROW):\n        for j in range(COL):\n            axes[i][j].imshow(generated_image[count])\n            axes[i][j].axis('off')\n            count += 1\n            \n    fig.savefig('generated_faces_dcgan/No.%d.png' % epoch)\n    plt.pyplot.close()\n    \n    #plt.image.imsave('images/'+str(epoch)+'.jpg')\n"

In [65]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Dense(512 , input_shape=(LATENT_DIM , )))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1 , activation='sigmoid'))
    
    encoder_feature = Input(shape=(LATENT_DIM , ))
    validity = model(encoder_feature)
    
    return Model(encoder_feature , validity , name='discriminator_Model')

In [66]:
def exp(x):
    return K.exp(x)

def encoder():
    image = Input(shape=(HEIGHT , WIDTH , CHANNEL))
    
    h = Flatten()(image)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    
    mean = Dense(LATENT_DIM)(h)
    log_var = Dense(LATENT_DIM)(h)
    
    a=Activation(exp)(log_var)
    b=K.random_normal(shape=K.shape(log_var))
    encoder_feature = Add()([mean , Multiply()([a , b])])
    #    
    return Model(image , a)

def decoder():
    input_feature = Input(shape=(LATENT_DIM , ))
    
    h = Dense(512 , input_shape=(LATENT_DIM , ))(input_feature)
    h = LeakyReLU(alpha=0.2)(h)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    h = Dense(HEIGHT*WIDTH*CHANNEL)(h)
    h = Activation('tanh')(h)
    image_hat = Reshape(target_shape=(HEIGHT , WIDTH , CHANNEL))(h)
    
    return Model(input_feature , image_hat)

In [73]:
adam = Adam(lr = 0.0002 , beta_1=0.5)

In [74]:
discriminator_i = discriminator()
discriminator_i.compile(optimizer=adam , loss='binary_crossentropy' , metrics=['accuracy'])


encoder_i = encoder()
decoder_i = decoder()

image = Input(shape=(HEIGHT , WIDTH , CHANNEL))

encoder_feature = encoder_i(image)
image_hat = decoder_i(encoder_feature)

discriminator_i.trainable = False

validity = discriminator_i(encoder_feature)

#aae为重新构造的组合model
aae = Model(image , [image_hat , validity])

aae.compile(optimizer=adam , loss=['mse' , 'binary_crossentropy'] , loss_weights = [0.999 , 0.001])

In [ ]:
real_labels = np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.zeros(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(10000):
    noise_encoder_feature = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))

    #real_image = load_image()
    real_image = load_mnist()
    #训练判别器
    real_encoder_feature = encoder_i.predict(real_image)

    noise_loss = discriminator_i.train_on_batch(noise_encoder_feature , real_labels)
    real_loss = discriminator_i.train_on_batch(real_encoder_feature , fake_labels)

    loss = np.add(noise_loss , real_loss)/2

    #训练aae
    aae_loss = aae.train_on_batch(real_image , [real_image , real_labels])
    
    print('epoch:%d loss:%f accu:%f aae_loss_1:%f aae_loss_2:%f' % (i , loss[0] , loss[1] , aae_loss[0] , aae_loss[1]))
    
    if i % 200 == 0:
        #write_image(i)
        write_image_mnist(i)
    
#write_image(999)
write_image_mnist(999)


C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 loss:16.019650 accu:0.000000 aae_loss_1:0.015980 aae_loss_2:0.015996
epoch:1 loss:14.639118 accu:0.000000 aae_loss_1:0.016262 aae_loss_2:0.016278
epoch:2 loss:9.329687 accu:0.136719 aae_loss_1:0.015477 aae_loss_2:0.015492
epoch:3 loss:7.971489 accu:0.500000 aae_loss_1:0.015405 aae_loss_2:0.015420
epoch:4 loss:7.971227 accu:0.500000 aae_loss_1:0.014602 aae_loss_2:0.014617
epoch:5 loss:7.971199 accu:0.500000 aae_loss_1:0.014125 aae_loss_2:0.014139
epoch:6 loss:7.971213 accu:0.500000 aae_loss_1:0.013097 aae_loss_2:0.013110
epoch:7 loss:7.971198 accu:0.500000 aae_loss_1:0.013589 aae_loss_2:0.013602
epoch:8 loss:7.971196 accu:0.500000 aae_loss_1:0.012287 aae_loss_2:0.012300
epoch:9 loss:7.971204 accu:0.500000 aae_loss_1:0.013292 aae_loss_2:0.013306
epoch:10 loss:7.971231 accu:0.500000 aae_loss_1:0.013069 aae_loss_2:0.013082
epoch:11 loss:7.971200 accu:0.500000 aae_loss_1:0.012406 aae_loss_2:0.012418
epoch:12 loss:7.971199 accu:0.500000 aae_loss_1:0.012928 aae_loss_2:0.012941
epoch:1

epoch:112 loss:7.971197 accu:0.500000 aae_loss_1:0.015026 aae_loss_2:0.015041
epoch:113 loss:7.971199 accu:0.500000 aae_loss_1:0.013573 aae_loss_2:0.013587
epoch:114 loss:7.971197 accu:0.500000 aae_loss_1:0.013246 aae_loss_2:0.013259
epoch:115 loss:7.971197 accu:0.500000 aae_loss_1:0.012431 aae_loss_2:0.012443
epoch:116 loss:7.971199 accu:0.500000 aae_loss_1:0.013431 aae_loss_2:0.013445
epoch:117 loss:7.971198 accu:0.500000 aae_loss_1:0.012520 aae_loss_2:0.012532
epoch:118 loss:7.971200 accu:0.500000 aae_loss_1:0.012916 aae_loss_2:0.012928
epoch:119 loss:7.971200 accu:0.500000 aae_loss_1:0.013881 aae_loss_2:0.013895
epoch:120 loss:7.971204 accu:0.500000 aae_loss_1:0.013530 aae_loss_2:0.013544
epoch:121 loss:7.971195 accu:0.500000 aae_loss_1:0.015154 aae_loss_2:0.015169
epoch:122 loss:7.971199 accu:0.500000 aae_loss_1:0.013674 aae_loss_2:0.013688
epoch:123 loss:7.971196 accu:0.500000 aae_loss_1:0.014734 aae_loss_2:0.014749
epoch:124 loss:7.971198 accu:0.500000 aae_loss_1:0.013670 aae_lo

epoch:223 loss:7.971201 accu:0.500000 aae_loss_1:0.014890 aae_loss_2:0.014905
epoch:224 loss:7.971200 accu:0.500000 aae_loss_1:0.014604 aae_loss_2:0.014619
epoch:225 loss:7.971198 accu:0.500000 aae_loss_1:0.016705 aae_loss_2:0.016721
epoch:226 loss:7.971198 accu:0.500000 aae_loss_1:0.017267 aae_loss_2:0.017284
epoch:227 loss:7.971200 accu:0.500000 aae_loss_1:0.015636 aae_loss_2:0.015652
epoch:228 loss:7.971201 accu:0.500000 aae_loss_1:0.014738 aae_loss_2:0.014752
epoch:229 loss:7.971201 accu:0.500000 aae_loss_1:0.014096 aae_loss_2:0.014110
epoch:230 loss:7.971199 accu:0.500000 aae_loss_1:0.013488 aae_loss_2:0.013501
epoch:231 loss:7.971197 accu:0.500000 aae_loss_1:0.014178 aae_loss_2:0.014192
epoch:232 loss:7.971196 accu:0.500000 aae_loss_1:0.013060 aae_loss_2:0.013073
epoch:233 loss:7.971200 accu:0.500000 aae_loss_1:0.013712 aae_loss_2:0.013725
epoch:234 loss:7.971199 accu:0.500000 aae_loss_1:0.014215 aae_loss_2:0.014229
epoch:235 loss:7.971206 accu:0.500000 aae_loss_1:0.013761 aae_lo

epoch:339 loss:7.971199 accu:0.500000 aae_loss_1:0.014452 aae_loss_2:0.014466
epoch:340 loss:7.971197 accu:0.500000 aae_loss_1:0.014784 aae_loss_2:0.014799
epoch:341 loss:7.971212 accu:0.500000 aae_loss_1:0.015562 aae_loss_2:0.015577
epoch:342 loss:7.971198 accu:0.500000 aae_loss_1:0.016732 aae_loss_2:0.016749
epoch:343 loss:7.971198 accu:0.500000 aae_loss_1:0.015579 aae_loss_2:0.015595
epoch:344 loss:7.971198 accu:0.500000 aae_loss_1:0.016219 aae_loss_2:0.016235
epoch:345 loss:7.971197 accu:0.500000 aae_loss_1:0.015448 aae_loss_2:0.015463
epoch:346 loss:7.971200 accu:0.500000 aae_loss_1:0.015617 aae_loss_2:0.015633
epoch:347 loss:7.971199 accu:0.500000 aae_loss_1:0.015226 aae_loss_2:0.015241
epoch:348 loss:7.971201 accu:0.500000 aae_loss_1:0.013246 aae_loss_2:0.013260
epoch:349 loss:7.971196 accu:0.500000 aae_loss_1:0.012904 aae_loss_2:0.012917
epoch:350 loss:7.971200 accu:0.500000 aae_loss_1:0.012069 aae_loss_2:0.012081
epoch:351 loss:7.971207 accu:0.500000 aae_loss_1:0.012892 aae_lo

epoch:447 loss:7.971198 accu:0.500000 aae_loss_1:0.014537 aae_loss_2:0.014552
epoch:448 loss:7.971198 accu:0.500000 aae_loss_1:0.013689 aae_loss_2:0.013702
epoch:449 loss:7.971199 accu:0.500000 aae_loss_1:0.014215 aae_loss_2:0.014230
epoch:450 loss:7.971204 accu:0.500000 aae_loss_1:0.013894 aae_loss_2:0.013907
epoch:451 loss:7.971197 accu:0.500000 aae_loss_1:0.014584 aae_loss_2:0.014598
epoch:452 loss:7.971199 accu:0.500000 aae_loss_1:0.014967 aae_loss_2:0.014982
epoch:453 loss:7.971198 accu:0.500000 aae_loss_1:0.012644 aae_loss_2:0.012657
epoch:454 loss:7.971200 accu:0.500000 aae_loss_1:0.013134 aae_loss_2:0.013147
epoch:455 loss:7.971198 accu:0.500000 aae_loss_1:0.012933 aae_loss_2:0.012946
epoch:456 loss:7.971198 accu:0.500000 aae_loss_1:0.013067 aae_loss_2:0.013080
epoch:457 loss:7.971200 accu:0.500000 aae_loss_1:0.013103 aae_loss_2:0.013116
epoch:458 loss:7.971196 accu:0.500000 aae_loss_1:0.014807 aae_loss_2:0.014822
epoch:459 loss:7.971198 accu:0.500000 aae_loss_1:0.013325 aae_lo

epoch:557 loss:7.971200 accu:0.500000 aae_loss_1:0.014979 aae_loss_2:0.014994
epoch:558 loss:7.971196 accu:0.500000 aae_loss_1:0.014030 aae_loss_2:0.014044
epoch:559 loss:7.971197 accu:0.500000 aae_loss_1:0.012417 aae_loss_2:0.012429
epoch:560 loss:7.971197 accu:0.500000 aae_loss_1:0.012936 aae_loss_2:0.012949
epoch:561 loss:7.971199 accu:0.500000 aae_loss_1:0.013323 aae_loss_2:0.013336
epoch:562 loss:7.971196 accu:0.500000 aae_loss_1:0.012970 aae_loss_2:0.012983
epoch:563 loss:7.971197 accu:0.500000 aae_loss_1:0.015065 aae_loss_2:0.015080
epoch:564 loss:7.971199 accu:0.500000 aae_loss_1:0.013734 aae_loss_2:0.013748
epoch:565 loss:7.971198 accu:0.500000 aae_loss_1:0.013238 aae_loss_2:0.013251
epoch:566 loss:7.971199 accu:0.500000 aae_loss_1:0.014011 aae_loss_2:0.014025
epoch:567 loss:7.971199 accu:0.500000 aae_loss_1:0.013761 aae_loss_2:0.013775
epoch:568 loss:7.971197 accu:0.500000 aae_loss_1:0.013456 aae_loss_2:0.013470
epoch:569 loss:7.971205 accu:0.500000 aae_loss_1:0.011938 aae_lo

epoch:667 loss:7.971200 accu:0.500000 aae_loss_1:0.013768 aae_loss_2:0.013781
epoch:668 loss:7.971200 accu:0.500000 aae_loss_1:0.012065 aae_loss_2:0.012077
epoch:669 loss:7.971204 accu:0.500000 aae_loss_1:0.013270 aae_loss_2:0.013283
epoch:670 loss:7.971199 accu:0.500000 aae_loss_1:0.013553 aae_loss_2:0.013567
epoch:671 loss:7.971198 accu:0.500000 aae_loss_1:0.014762 aae_loss_2:0.014776
epoch:672 loss:7.971197 accu:0.500000 aae_loss_1:0.015282 aae_loss_2:0.015298
epoch:673 loss:7.971198 accu:0.500000 aae_loss_1:0.014636 aae_loss_2:0.014651
epoch:674 loss:7.971196 accu:0.500000 aae_loss_1:0.014009 aae_loss_2:0.014023
epoch:675 loss:7.971195 accu:0.500000 aae_loss_1:0.014281 aae_loss_2:0.014295
epoch:676 loss:7.971195 accu:0.500000 aae_loss_1:0.013548 aae_loss_2:0.013561
epoch:677 loss:7.971196 accu:0.500000 aae_loss_1:0.015236 aae_loss_2:0.015251
epoch:678 loss:7.971196 accu:0.500000 aae_loss_1:0.015878 aae_loss_2:0.015894
epoch:679 loss:7.971198 accu:0.500000 aae_loss_1:0.013937 aae_lo

epoch:782 loss:7.971196 accu:0.500000 aae_loss_1:0.020689 aae_loss_2:0.020710
epoch:783 loss:7.971197 accu:0.500000 aae_loss_1:0.016689 aae_loss_2:0.016706
epoch:784 loss:7.971198 accu:0.500000 aae_loss_1:0.015382 aae_loss_2:0.015398
epoch:785 loss:7.971195 accu:0.500000 aae_loss_1:0.014049 aae_loss_2:0.014063
epoch:786 loss:7.971197 accu:0.500000 aae_loss_1:0.014026 aae_loss_2:0.014040
epoch:787 loss:7.971197 accu:0.500000 aae_loss_1:0.013283 aae_loss_2:0.013296
epoch:788 loss:7.971198 accu:0.500000 aae_loss_1:0.014384 aae_loss_2:0.014398
epoch:789 loss:7.971199 accu:0.500000 aae_loss_1:0.012576 aae_loss_2:0.012589
epoch:790 loss:7.971200 accu:0.500000 aae_loss_1:0.012770 aae_loss_2:0.012783
epoch:791 loss:7.971198 accu:0.500000 aae_loss_1:0.013476 aae_loss_2:0.013490
epoch:792 loss:7.971197 accu:0.500000 aae_loss_1:0.012964 aae_loss_2:0.012977
epoch:793 loss:7.971197 accu:0.500000 aae_loss_1:0.014001 aae_loss_2:0.014015
epoch:794 loss:7.971199 accu:0.500000 aae_loss_1:0.013472 aae_lo

In [70]:
real_labels.shape

(64, 1)

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [39]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 96, 96, 3)         29029120  
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 14320641  
Total params: 43,349,761
Trainable params: 29,025,536
Non-trainable params: 14,324,225
_________________________________________________________________


In [ ]:
gc.collect()

In [14]:
modeli = Sequential()

modeli.add(Dense(128 * 7 * 7, activation="relu", input_shape=(LATENT_DIM,)))
modeli.add(Reshape((7, 7, 128)))
modeli.add(UpSampling2D())
modeli.add(Conv2D(128, kernel_size=3, padding="same"))
modeli.add(BatchNormalization(momentum=0.8))
modeli.add(Activation("relu"))
modeli.add(UpSampling2D())
modeli.add(Conv2D(64, kernel_size=3, padding="same"))
modeli.add(BatchNormalization(momentum=0.8))
modeli.add(Activation("relu"))
modeli.add(Conv2D(CHANNEL, kernel_size=3, padding="same"))
modeli.add(Activation("tanh"))

In [15]:
modeli.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 28, 28, 128)       0         
__________

In [41]:
32*400

12800

In [ ]:
gc.collect()

In [30]:
aa=encoder()

AttributeError: 'NoneType' object has no attribute '_inbound_nodes'